In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import print_title, print_label
from utilities import save_data

In [2]:
# Import libraries for data analysis and visualization
import pandas as pd
import yfinance as yf # Yahoo Finance data retrieval

### Data Collection
___

In [3]:
# Fetch S&P 500 tickers

#Get tickers from wikipedia
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_numpy()

# Filter out Class B shares that have a '.B' in the ticker name
# Class B shares are typically held by company insiders and have different voting rights than Class A shares
sp500_tickers = [ticker for ticker in sp500_tickers if '.B' not in ticker]

# Total Tickers
print_title("`S&P_500` total tickers: " + str(len(sp500_tickers)))

╔═══════════════════════════════════════════════════════════════╗
║                 `S&P_500` total tickers: 501                  ║
╚═══════════════════════════════════════════════════════════════╝


In [4]:
# Define dates for historical data download
start_date = '2000-01-01'
end_date = '2024-10-14'

try:
    # Download historical prices
    data = yf.download(sp500_tickers, start=start_date, end=end_date)

    # Check if data was successfully downloaded
    downloaded_tickers = data.columns.get_level_values(0).unique()
    print_title(
        f"Successfully downloaded tickers: {len(data.columns.get_level_values(1).unique())} out of {len(sp500_tickers)}", "bright_green", "green")

except Exception as e:
    # Handle any errors that occur during the download
    print_title(
        f"Failed to download data: {str(e)}", "bright_red", "red")

[*********************100%***********************]  501 of 501 completed


╔═══════════════════════════════════════════════════════════════╗
║        Successfully downloaded tickers: 501 out of 501        ║
╚═══════════════════════════════════════════════════════════════╝


### Data Cleaning

In [5]:
# Inspect current dataframe structure
print("Shape:", data.shape)
data.head()

Shape: (6234, 3006)


Price                      Adj Close                                          \
Ticker                             A      AAPL ABBV ABNB       ABT      ACGL   
Date                                                                           
2000-01-03 00:00:00+00:00  43.463024  0.844004  NaN  NaN  8.288177  1.277778   
2000-01-04 00:00:00+00:00  40.142941  0.772846  NaN  NaN  8.051375  1.270833   
2000-01-05 00:00:00+00:00  37.652863  0.784155  NaN  NaN  8.036574  1.388889   
2000-01-06 00:00:00+00:00  36.219193  0.716296  NaN  NaN  8.317777  1.375000   
2000-01-07 00:00:00+00:00  39.237457  0.750226  NaN  NaN  8.406583  1.451389   

Price                                                          ... Volume  \
Ticker                    ACN       ADBE        ADI       ADM  ...    WTW   
Date                                                           ...          
2000-01-03 00:00:00+00:00 NaN  16.274672  28.214649  6.307345  ...    NaN   
2000-01-04 00:00:00+00:00 NaN  14.909399  26.787287  6.241642  ...    NaN   
2000-01-05 00:00:00+00:00 NaN  15.204173  27.178352  6.143092  ...    NaN   
2000-01-06 00:00:00+00:00 NaN  15.328288  26.435343  6.175944  ...    NaN   
2000-01-07 00:00:00+00:00 NaN  16.072983  27.178352  6.274496  ...    NaN   

Price                                                                        \
Ticker                          WY WYNN      XEL       XOM XYL      YUM ZBH   
Date                                                                          
2000-01-03 00:00:00+00:00   973700  NaN  2738600  13458200 NaN  3033493 NaN   
2000-01-04 00:00:00+00:00  1201700  NaN   425200  14510800 NaN  3315031 NaN   
2000-01-05 00:00:00+00:00  1184600  NaN   500200  17485000 NaN  4642602 NaN   
2000-01-06 00:00:00+00:00  1307700  NaN   344100  19461600 NaN  3947658 NaN   
2000-01-07 00:00:00+00:00  1728000  NaN   469500  16603800 NaN  6063647 NaN   

Price                                   
Ticker                        ZBRA ZTS  
Date                                    
2000-01-03 00:00:00+00:00  1055700 NaN  
2000-01-04 00:00:00+00:00   522450 NaN  
2000-01-05 00:00:00+00:00   612225 NaN  
2000-01-06 00:00:00+00:00   263925 NaN  
2000-01-07 00:00:00+00:00   333900 NaN  

[5 rows x 3006 columns]

In [6]:
# Remove `Close`, `High`, `Low`, `Open`, `Volume` columns
# Keep only `Adj Close` for each ticker

# The closing price adjusted for corporate actions like stock splits and dividends. 
# It’s the most accurate reflection of the stock’s value for historical analysis.
data_clean = data['Adj Close']

print("Shape:", data_clean.shape)
data_clean.head()

Shape: (6234, 501)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03 00:00:00+00:00,43.463024,0.844004,NaN,NaN,8.288177,1.277778,NaN,16.274672,28.214649,6.307345,...,NaN,11.352943,NaN,6.848377,18.035816,NaN,4.634288,NaN,25.027779,NaN
2000-01-04 00:00:00+00:00,40.142941,0.772846,NaN,NaN,8.051375,1.270833,NaN,14.909399,26.787287,6.241642,...,NaN,10.926451,NaN,7.006068,17.690357,NaN,4.541133,NaN,24.666668,NaN
2000-01-05 00:00:00+00:00,37.652863,0.784155,NaN,NaN,8.036574,1.388889,NaN,15.204173,27.178352,6.143092,...,NaN,11.505273,NaN,7.276400,18.654766,NaN,4.564423,NaN,25.138889,NaN
2000-01-06 00:00:00+00:00,36.219193,0.716296,NaN,NaN,8.317777,1.375000,NaN,15.328288,26.435343,6.175944,...,NaN,12.043463,NaN,7.208816,19.619175,NaN,4.525609,NaN,23.777779,NaN
2000-01-07 00:00:00+00:00,39.237457,0.750226,NaN,NaN,8.406583,1.451389,NaN,16.072983,27.178352,6.274496,...,NaN,11.647437,NaN,7.208816,19.561605,NaN,4.424695,NaN,23.513889,NaN


### Impute missing values (NA's)

In [7]:
# Check for missing values in each column

print_title("Missing values in each column", closed_corners=False)
missing_values = data_clean.isnull().sum()

for index, value in missing_values.items():
    if value > 0:
        print_label(f"{index}", f"{value}")


╔═══════════════════════════════════════════════════════════════╗
║                 Missing values in each column                 ║
╠═══════════════════════════════════════════════════════════════╣
║ ABBV                           |             3269             ║
║ ABNB                           |             5269             ║
║ ACN                            |             389              ║
║ AIZ                            |             1027             ║
║ ALGN                           |             271              ║
║ ALLE                           |             3491             ║
║ AMCR                           |             3111             ║
║ AMP                            |             1433             ║
║ AMTM                           |             6220             ║
║ ANET                           |             3628             ║
║ APTV                           |             2989             ║
║ AVGO                           |             2412             ║
║ AWK     

In [8]:
# Impute missing values with 0

# Fill missing values with 0
data_clean = data_clean.fillna(0)

# Confirm that there are no missing values
print("Number of NA's:", data_clean.isnull().sum().sum())

Number of NA's: 0


### Save dataset

In [9]:
# Export to a new CSV file
file_name = f"sp500_adj_close_{start_date}_{end_date}.csv"
file_path = f"../../../data/raw_data/{file_name}"

save_data(data, file_path)

╔═══════════════════════════════════════════════════════════════╗
║   File saved as `sp500_adj_close_2000-01-01_2024-10-14.csv`   ║
╚═══════════════════════════════════════════════════════════════╝
